In [ ]:
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster, MousePosition, Fullscreen, MiniMap
from shapely import geometry

# Импортируем исходные данные: административная граница городского округа, полигоны зданий и точечный слой с ОКН
adm_border = gpd.read_file('/Users/kirilltrusov/Desktop/Folium-map-Tula-OKN--main-2/adm_border.geojson')
okn_points = gpd.read_file('/Users/kirilltrusov/Desktop/Folium-map-Tula-OKN--main-2/OKN_points1.geojson')
buildings_polygons = gpd.read_file('/Users/kirilltrusov/Desktop/Folium-map-Tula-OKN--main-2/buildings_polygons.geojson')

# Настраиваем проекцию в зависимости от зоны UTM города
adm_border = adm_border.to_crs("EPSG:32637")
okn_points = okn_points.to_crs("EPSG:32637")
buildings_polygons = buildings_polygons.to_crs("EPSG:32637")

# Удаляем лишние атрибуты в слое ОКН
okn_points = okn_points[['field_1', 'Объект', 'Полный адрес', 'Категория историко-культурного значения', 'Вид объекта', 'Принадлежность к Юнеско', 'Особо ценный объект', 'дата создания', 'geometry']]

# Присоединяем информацию об ОКН к зданиям
buildings_polygons.crs = okn_points.crs
buildings_polygons = buildings_polygons.sjoin(okn_points)

# Перепроецируем точки ОКН в проекцию EPSG:3857
okn_points = okn_points.to_crs("EPSG:3857")

# Создаем интерактивную карту
main_map = folium.Map(location=[okn_points.centroid.y.mean(), okn_points.centroid.x.mean()], zoom_start=12,  tiles="cartodb positron", control_scale=True)

# Настраиваем и добавляем слой с административной границей гороского округа Тула
style1 = {'fillColor': 'none', 'color': 'red', 'weight': 2}
folium.GeoJson(
    adm_border,
    name="Граница городского округа Тула",
    style_function=lambda x: style1,
    highlight_function=lambda x: {"fillOpacity": 0.8},
    zoom_on_click=True,
    show=False,
).add_to(main_map)

# Настраиваем и добавляем слой с объектами культурного наследия
style2 = {'fillColor': 'yellow', 'color': 'yellow', 'weight': 1}
folium.GeoJson(
    buildings_polygons,
    name="Объекты культурного наследия",
    style_function=lambda x: style2,
    highlight_function=lambda x: {"fillOpacity": 0.5},
    zoom_on_click=True,
    show=False,
).add_to(main_map)

# Добавляем кластеры точек ОКН на карту
marker_cluster = MarkerCluster().add_to(main_map)
for idx, row in okn_points.iterrows():
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup=row['Объект'],
    ).add_to(marker_cluster)

# Добавляем инструменты управления картой
MousePosition().add_to(main_map)
Fullscreen().add_to(main_map)
MiniMap().add_to(main_map)

main_map